# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
import statsmodels
from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [2]:
# Run this code:
salaries = pd.read_csv(r'Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [4]:
from scipy.stats import ttest_ind, norm, ks_2samp

In [5]:
salaries['Hourly Rate'].mean()

32.78855771628023

In [12]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [6]:
# Initialize variables
contador = 0  # This will track the fraction of the dataset allocated to the test group
data = {}  # To store results

# Loop to calculate MDE for splits ranging from 5% to 50% of the dataset
while contador <= 0.45:
    contador += 0.05  # Increment the fraction for each iteration
    mde_list = []  # To store MDE values for each iteration
    avg_list = []  # To store average values for the control group
    
    # Perform 30 simulations for each split to average the MDE
    for i in range(1, 31):
        # Randomly split the dataset into test and control groups based on the current fraction
        test_df, control_df = train_test_split(salaries, test_size=contador)
        
        # Define the parameters for the MDE calculation
        alpha = 0.05  # Significance level
        beta = 0.95    # Desired statistical power
        
        # Calculate standard deviations for both groups
        std_dev_test = np.std(test_df['Hourly Rate'])
        std_dev_control = np.std(control_df['Hourly Rate'])
        
        # Calculate sample sizes
        sample_size_control = len(control_df)
        sample_size_test = len(test_df)
        
        # Calculate critical values for the two-tailed test
        z_alpha = norm.ppf(1 - alpha/2)
        z_beta = norm.ppf(beta)
        
        # Calculate pooled standard deviation
        pooled_std_dev = np.sqrt(((sample_size_control - 1) * std_dev_control**2 + 
                                  (sample_size_test - 1) * std_dev_test**2) / 
                                 (sample_size_control + sample_size_test - 2))
        
        # Calculate the MDE
        mde = (z_alpha + z_beta) * pooled_std_dev * np.sqrt(2 / sample_size_control)
        
        # Append the MDE and average control group price to their respective lists
        mde_list.append(mde)
        avg = np.mean(control_df['Hourly Rate'])
        avg_list.append(avg)
    
    # Calculate and store the average MDE, average price, and MDE percentage for the current split
    data[(round(contador, 2))] = {'mde': round(np.mean(mde_list), 4),
                                  'avg': round(np.mean(avg_list), 4),
                                  'mde_%': round(np.mean(mde_list) / np.mean(avg_list) * 100, 2)}

In [7]:
data

{0.05: {'mde': 1.5153, 'avg': 32.8776, 'mde_%': 4.61},
 0.1: {'mde': 1.0717, 'avg': 32.9317, 'mde_%': 3.25},
 0.15: {'mde': 0.8751, 'avg': 32.7632, 'mde_%': 2.67},
 0.2: {'mde': 0.7579, 'avg': 32.8393, 'mde_%': 2.31},
 0.25: {'mde': 0.6778, 'avg': 32.8033, 'mde_%': 2.07},
 0.3: {'mde': 0.6188, 'avg': 32.7327, 'mde_%': 1.89},
 0.35: {'mde': 0.5729, 'avg': 32.7114, 'mde_%': 1.75},
 0.4: {'mde': 0.5359, 'avg': 32.7938, 'mde_%': 1.63},
 0.45: {'mde': 0.5053, 'avg': 32.7745, 'mde_%': 1.54},
 0.5: {'mde': 0.4793, 'avg': 32.7501, 'mde_%': 1.46}}

El salario medio de cada tramo de test es superior a 30$/h.

# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [8]:
import numpy as np
from scipy import stats

mean_hourly_rate = np.mean(test_df['Hourly Rate'])
standard_error = stats.sem(test_df['Hourly Rate'])
confidence_level = 0.95
degrees_freedom = len(test_df['Hourly Rate']) - 1

confidence_interval = stats.t.interval(confidence_level, 
                                       degrees_freedom, 
                                       loc = mean_hourly_rate, 
                                       scale = standard_error)

print(f"Intervalo de confianza del 95%: {confidence_interval}")

Intervalo de confianza del 95%: (nan, nan)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [13]:
import statsmodels.api as sm

# Número de trabajadores por hora en Chicago
count = 8022  

# Tamaño total de la muestra
nobs = 33183 

# Proporción bajo la hipótesis nula
prop_null = 0.25

# Realizar la prueba z
z_stat, p_value = sm.stats.proportions_ztest(count, nobs, value=prop_null, alternative='two-sided')

# Mostrar los resultados
print(f"Estadístico z: {z_stat:.2f}")
print(f"Valor p: {p_value:.3f}")

# Interpretación del resultado
alpha = 0.05  # Nivel de significancia del 5%

if p_value < alpha:
    print("Rechazamos la hipótesis nula: El porcentaje de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No podemos rechazar la hipótesis nula: No hay evidencia suficiente de que el porcentaje de trabajadores por hora sea diferente del 25%.")

Estadístico z: -3.51
Valor p: 0.000
Rechazamos la hipótesis nula: El porcentaje de trabajadores por hora es significativamente diferente del 25%.
